# ETL Pipeline

The Goal of this script is to create a non-relational database and ETL pipeline for a music streaming app. 
Its tables should be able to answer queries with in the following form:   
* Give me the artist, song title and song's length in the music app history that was heard   
during  sessionId = 338, and itemInSession  = 4
* Give me only the following: name of artist, song (sorted by itemInSession) and user (first   
and last name) for userid = 10, sessionid = 182
* Give me every user name (first and last) in my music app history who listened to the song    
'All Hands Against His Own'   


## #1 Preparing Files

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from typing import Iterator, Dict, Any, Optional

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
#print(len(full_data_rows_list))

# preview list of data rows
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## #2 Preparing Keyspace

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)
    
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Handling queries efficiently

In [6]:
def cql(query: str) -> None:
    """Query handling"""
    try:
        session.execute(query)
    except Exception as e:
        print(e)

## #3 Creating Tables and Processing Data

In the following sections I will design a table to answer one of the queries mentioned at the beginning.
Therefore the following steps apply:
* translating the questions into cql
* designing tables on the basis of those queries
* inserting the data from csv into those tables
* validating the data by answering the example tasks mentioned above.


### Query1
Task: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Translation in to CQL:
* SELECT artist, song, length FROM table1 WHERE session_id = 338 AND item_in_session = 4

In [29]:
# create table1
cql("""DROP TABLE IF EXISTS songs; """)
cql("""CREATE TABLE songs (
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    length DOUBLE,
    PRIMARY KEY ((sessionId, itemInSession))
    )""")



# insert data into table1
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO songs (sessionId, itemInSession, artist, song, length)
                VALUES (%s, %s, %s, %s, %s)"""
        #Each row read from the csv file is returned as a list of strings No automatic data type conversion is performed
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

        
    
# validate data
#code integrated from here: https://stackoverflow.com/a/41484806/13560354
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

## The format to return row results. By default, each returned row will be a named tuple.
session.row_factory = pandas_factory

##By default, this many rows will be fetched at a time. Setting this to None will disable automatic paging for large query results
session.default_fetch_size = None

query = "SELECT * FROM songs WHERE sessionId = 338 AND itemInSession = 4 LIMIT 10"
results = session.execute(query, timeout=None)

##The list of current page rows. May be empty if the result was empty, or this is the last page.
df1 = results._current_rows
df1[['artist', 'song', 'length']].head()



,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### Query2
Task: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Translation in to CQL:
* SELECT artist, song, first_name, last_name FROM X WHERE session_id = 182 AND user_id = 10

In [24]:
# create table2
cql("""DROP TABLE IF EXISTS songplays; """)
cql("""CREATE TABLE table2 (
    sessionId INT,
    itemInSession INT,
    userId INT,
    artist TEXT,
    song TEXT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY ((sessionId, userId), itemInSession))
    """)


# insert data into table2
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO songplays (sessionId, itemInSession, userId, artist, song, firstName, lastName)
                VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], line[1], line[4]))

        
# validate data
session.row_factory = pandas_factory
session.default_fetch_size = None
query = "SELECT * FROM songplays WHERE sessionId = 182 AND userId = 10"
results = session.execute(query, timeout=None)
df2 = results._current_rows
df2[['artist', 'song', 'firstname', 'lastname']].head()


,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query3
Task: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Translation in to CQL:
* SELECT first_name, last_name FROM X WHERE song = 'All Hands Against His Own'

In [25]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

cql("""DROP TABLE IF EXISTS users; """)
cql("""CREATE TABLE table3 (
    song TEXT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY (song, lastName, firstName)
    )""")

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO users (song, lastName, firstName)
                VALUES (%s, %s, %s)"""
        session.execute(query, (line[9], line[1], line[4]))

# validate data
session.row_factory = pandas_factory
session.default_fetch_size = None
query = "SELECT * FROM users WHERE song = 'All Hands Against His Own' "
results = session.execute(query, timeout=None)
df3 = results._current_rows
df3[['firstname', 'lastname']].head()


,firstname,lastname
0,Lynch,Jacqueline
1,Johnson,Sara
2,Levine,Tegan


### Dropping the tables and closing the session

In [68]:
cql("""DROP TABLE IF EXISTS songs; """)
cql("""DROP TABLE IF EXISTS songplays; """)
cql("""DROP TABLE IF EXISTS users; """)
session.shutdown()
cluster.shutdown()